# Analysis

### Reading in the data

In [0]:
# load the data
airline_df = spark.read.format("delta").load("dbfs:/user/airline/table")

In [0]:
from pyspark.sql.types import IntegerType, FloatType, DateType
import pyspark.sql.functions as F

In [0]:
# convert numeric columns to int/float
new_df = airline_df.withColumn("FL_DATE", airline_df.FL_DATE.cast(DateType())) \
                   .withColumn("OP_CARRIER_FL_NUM", airline_df.OP_CARRIER_FL_NUM.cast(IntegerType())) \
                   .withColumn("CRS_DEP_TIME", airline_df.CRS_DEP_TIME.cast(IntegerType())) \
                   .withColumn("DEP_TIME", airline_df.DEP_TIME.cast(FloatType())) \
                   .withColumn("DEP_DELAY", airline_df.DEP_DELAY.cast(FloatType())) \
                   .withColumn("TAXI_OUT", airline_df.TAXI_OUT.cast(FloatType())) \
                   .withColumn("WHEELS_OFF", airline_df.WHEELS_OFF.cast(FloatType())) \
                   .withColumn("WHEELS_ON", airline_df.WHEELS_ON.cast(FloatType())) \
                   .withColumn("TAXI_IN", airline_df.TAXI_IN.cast(FloatType())) \
                   .withColumn("CRS_ARR_TIME", airline_df.CRS_ARR_TIME.cast(IntegerType())) \
                   .withColumn("ARR_TIME", airline_df.ARR_TIME.cast(FloatType())) \
                   .withColumn("ARR_DELAY", airline_df.ARR_DELAY.cast(FloatType())) \
                   .withColumn("CANCELLED", airline_df.CANCELLED.cast(FloatType())) \
                   .withColumn("DIVERTED", airline_df.DIVERTED.cast(FloatType())) \
                   .withColumn("CRS_ELAPSED_TIME", airline_df.CRS_ELAPSED_TIME.cast(FloatType())) \
                   .withColumn("ACTUAL_ELAPSED_TIME", airline_df.ACTUAL_ELAPSED_TIME.cast(FloatType())) \
                   .withColumn("AIR_TIME", airline_df.AIR_TIME.cast(FloatType())) \
                   .withColumn("DISTANCE", airline_df.DISTANCE.cast(FloatType())) \
                   .withColumn("CARRIER_DELAY", airline_df.CARRIER_DELAY.cast(FloatType())) \
                   .withColumn("WEATHER_DELAY", airline_df.WEATHER_DELAY.cast(FloatType())) \
                   .withColumn("NAS_DELAY", airline_df.NAS_DELAY.cast(FloatType())) \
                   .withColumn("SECURITY_DELAY", airline_df.SECURITY_DELAY.cast(FloatType())) \
                   .withColumn("LATE_AIRCRAFT_DELAY", airline_df.LATE_AIRCRAFT_DELAY.cast(FloatType()))

### Downloading the airline code names

In [0]:
# airline name - code data from https://github.com/beanumber/airlines (originally from https://www.bts.gov/topics/airlines-and-airports/airline-codes)

!wget https://raw.githubusercontent.com/beanumber/airlines/master/data-raw/airlines.csv

--2022-05-30 08:32:55--  https://raw.githubusercontent.com/beanumber/airlines/master/data-raw/airlines.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46823 (46K) [text/plain]
Saving to: ‘airlines.csv’

airlines.csv        100%[===================>]  45.73K  --.-KB/s    in 0.009s  

2022-05-30 08:32:56 (5.01 MB/s) - ‘airlines.csv’ saved [46823/46823]



### Top 10 airlines with the most flight operations from 2009 to 2015

In [0]:
airline_names = spark.read.format("csv").option("header", "true").load("file:/databricks/driver/airlines.csv")

top_airlines = (new_df.groupby('OP_CARRIER').count().orderBy(F.desc('count'))
        .join(airline_names, airline_names.Code ==  new_df.OP_CARRIER, "left")
        .select(
            F.col('OP_CARRIER').alias('Airline carrier code'), 
            F.col('Description').alias('Airline carrier name'), 
            F.col('count').alias('Operations')
        )
        .take(10))

display(top_airlines)

Airline carrier code,Airline carrier name,Operations
WN,Southwest Airlines Co.,8115100
DL,Delta Air Lines Inc.,5046291
OO,SkyWest Airlines Inc.,4176455
AA,American Airlines Inc.,3954500
EV,ExpressJet Airlines Inc.,3693024
UA,United Air Lines Inc.,3075545
MQ,Envoy Air,2920259
US,US Airways Inc.,2657286
B6,JetBlue Airways,1597832
FL,AirTran Airways Corporation,1218389


Top 10 airlines with the most flight operations from 2009 to 2015 as a bar chart.

In [0]:
display(top_airlines)

Airline carrier code,Airline carrier name,Operations
WN,Southwest Airlines Co.,8115100
DL,Delta Air Lines Inc.,5046291
OO,SkyWest Airlines Inc.,4176455
AA,American Airlines Inc.,3954500
EV,ExpressJet Airlines Inc.,3693024
UA,United Air Lines Inc.,3075545
MQ,Envoy Air,2920259
US,US Airways Inc.,2657286
B6,JetBlue Airways,1597832
FL,AirTran Airways Corporation,1218389


### The proportion for the total flight cancellation reasons across 2009 to 2015

In [0]:
code_to_reason = {"A": "Airline/Carrier", "B": "Weather", "C": "National Air System", "D": "Security"}

display(new_df.groupby('CANCELLATION_CODE').count().orderBy(F.desc('count')).na.drop("any")
       .replace(to_replace=code_to_reason, subset=['CANCELLATION_CODE']).withColumnRenamed('CANCELLATION_CODE', 'Cancellation reason'))

Cancellation reason,count
Weather,331538
Airline/Carrier,247085
National Air System,129129
Security,319


Visualising it as a time series:

In [0]:
display(new_df.select('FL_DATE', 'CANCELLATION_CODE').na.drop("any").groupBy(['FL_DATE', 'CANCELLATION_CODE']).count().orderBy(F.asc('FL_DATE'))
       .replace(to_replace=code_to_reason, subset=['CANCELLATION_CODE'])
       .withColumnRenamed('CANCELLATION_CODE', 'Cancellation reason')
       .withColumnRenamed('count', 'Cancelled flights')
       .withColumnRenamed('FL_DATE', 'Date'))

Date,Cancellation reason,Cancelled flights
2009-01-01,Weather,63
2009-01-01,National Air System,1
2009-01-01,Airline/Carrier,94
2009-01-02,National Air System,27
2009-01-02,Airline/Carrier,117
2009-01-02,Weather,69
2009-01-03,Weather,106
2009-01-03,Airline/Carrier,83
2009-01-03,National Air System,68
2009-01-04,National Air System,31
